In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [24]:
data=pd.read_csv('phishing_email.csv')

In [25]:
data.head()

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [26]:
data.isnull().sum()

Unnamed: 0     0
Email Text    16
Email Type     0
dtype: int64

In [8]:
len(data)

18650

In [28]:
data_copy = data.copy()
data_copy = data_copy.drop('Unnamed: 0', axis=1)
data_copy

,Email Text,Email Type
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,software at incredibly low prices ( 86 % lower...,Phishing Email
...,...,...
18645,date a lonely housewife always wanted to date ...,Phishing Email
18646,request submitted : access request for anita ....,Safe Email
18647,"re : important - prc mtg hi dorn & john , as y...",Safe Email
18648,press clippings - letter on californian utilit...,Safe Email


In [29]:
data_copy.dropna(subset = ['Email Text'], inplace=True)

In [30]:
data_copy.isnull().sum()

Email Text    0
Email Type    0
dtype: int64

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(data_copy['Email Text'],  data_copy['Email Type'], test_size=0.2)

In [34]:
print(X_train.isnull().sum())

0


In [36]:
# X_train = X_train.fillna('')
# X_test = X_test.fillna('')
# print(X_train.type())
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18634 entries, 0 to 18649
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Email Text  18634 non-null  object
 1   Email Type  18634 non-null  object
dtypes: object(2)
memory usage: 436.7+ KB


In [38]:
X_train = X_train.astype(str)
X_test = X_test.astype(str)



from collections import Counter

print(f"Training class distributions summary: {Counter(Y_train)}")
print(f"Test class distributions summary: {Counter(Y_test)}")

Training class distributions summary: Counter({'Safe Email': 9027, 'Phishing Email': 5880})
Test class distributions summary: Counter({'Safe Email': 2295, 'Phishing Email': 1432})


In [40]:
!pip install imblearn

In [43]:
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline


model = make_pipeline_imb(TfidfVectorizer(), RandomUnderSampler(), MultinomialNB())
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)

In [45]:
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(Y_test, y_pred))

                      pre       rec       spe        f1       geo       iba       sup

Phishing Email       0.95      0.97      0.97      0.96      0.97      0.94      1432
    Safe Email       0.98      0.97      0.97      0.98      0.97      0.94      2295

   avg / total       0.97      0.97      0.97      0.97      0.97      0.94      3727



In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the email text to numerical features
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Define the model
model = KNeighborsClassifier()

# Fit the model with the vectorized data
model.fit(X_train_vectorized, Y_train)

KNeighborsClassifier()

In [47]:
#Проблеми со нерамнотежата на податоците: Печатење на статистика за распределбата на класите во обуката и тест податоците 
#покажува дали постои нерамнотежа помеѓу двата типа е-пошта (што може да има влијание на резултатите):
from collections import Counter

print(f"Training class distributions summary: {Counter(Y_train)}")
print(f"Test class distributions summary: {Counter(Y_test)}")

Training class distributions summary: Counter({'Safe Email': 9079, 'Phishing Email': 5841})
Test class distributions summary: Counter({'Safe Email': 2243, 'Phishing Email': 1487})


In [49]:
!pip install scikeras

In [ ]:
#Набрзо се додава и неуронска мрежа со Keras преку scikeras. 
#Oваа мрежа има Dense слоеви, Dropout слоеви за редукција на прекумерно прилагодување и sigmoid 
#активирачка функција за бинара класификација („phishing“ или „non-phishing“).
#Имплементација на неуронска мрежа

In [62]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from scikeras.wrappers import KerasClassifier
from imblearn.metrics import classification_report_imbalanced

# Custom transformer to convert sparse matrix to dense matrix
class DenseTransformer(BaseEstimator, TransformerMixin):
    def transform(self, X, y=None):
        return X.toarray()  # Convert sparse matrix to dense
    def fit(self, X, y=None):
        return self  # No fitting required for this transformer

# Define the function to create a Keras model
def create_model():
    model = Sequential()
    model.add(Input(shape=(1000,)))  # Define the input shape explicitly using Input layer
    model.add(Dense(128, activation='relu'))  # No need for input_dim here anymore
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define the neural network model wrapped as a classifier
nn_model = KerasClassifier(model=create_model, epochs=10, batch_size=32, verbose=1)

# Create a pipeline that first vectorizes the text, then converts it to dense, and finally fits the model
model = make_pipeline(
    TfidfVectorizer(max_features=1000),  # Convert text to features (1000 features)
    DenseTransformer(),  # Convert the sparse matrix to a dense matrix
    #DenseTransformer е корисен за преобразба на векторите од скрени матрици (кои се враќаат по векторизацијата) 
    #во густи матрици (dense matrices) кои не се ретки.
    nn_model  # Pass the transformed features to the neural network model
)

# Train the model
model.fit(X_train, Y_train)

# Make predictions
y_pred = model.predict(X_test)

# Print the classification report (imbalanced version for better performance evaluation)
print(classification_report_imbalanced(Y_test, y_pred))


Epoch 1/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8604 - loss: 0.2988
Epoch 2/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9691 - loss: 0.0815
Epoch 3/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9746 - loss: 0.0648
Epoch 4/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9806 - loss: 0.0504
Epoch 5/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9848 - loss: 0.0379
Epoch 6/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9868 - loss: 0.0302
Epoch 7/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9874 - loss: 0.0261
Epoch 8/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9884 - loss: 0.0240
Epoch 9/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9891 - loss: 0.0216
Epoch 10/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9891 - loss: 0.0209
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
                      pre       rec       spe        f1       geo       iba       sup

Phishing Em